In [2]:
# DATAFRAMES INITIALISATION

import os
os.chdir('C:\\Users\\asus\\OneDrive\\Documenti\\University Docs\\MSc Computing\\Final Project\\RainbowFood(JN)\\Rainbow-Food-Collaborative-Filtering-')

import pandas as pd

# vegetables file
col_list_veg = ["Vegetables", "Serving", "Calories"]
df_veg = pd.read_csv("Vegetables.csv", usecols = col_list_veg)

# allergies file
col_list_all = ["Class", "Type", "Group", "Food", "Allergy"]
df_all = pd.read_csv("FoodAllergies.csv", usecols = col_list_all)

# got rid of the Nan values because it considered as it was float instead of string (could not apply lower case)
df_all.dropna(inplace = True)

# recipe file
col_list_rec = ['Link', 'Title', 'Total Time', 'Servings', 'Ingredients', 'Instructions']
df_rec = pd.read_csv("Recipes.csv", usecols = col_list_rec)

# ratings
col_list_rat = ["userId", "recipeId", "rating"]
df_rat = pd.read_csv("Ratings_small.csv", usecols = col_list_rat)

# NLP FOR CLEANING UP INPUTS
# lemmatising
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# FUNCTIONS
# function to extract lists from pandas columns
def list_maker(column):
    return column.tolist()

# function to make lower case in lists
def lower_case(column_list):
    for x in range(len(column_list)):
        column_list[x] = column_list[x].lower()
    return column_list

# function to cut duplicates from a list
def no_duplicates(column_list):
        no_duplicates_list = []
        for x in column_list:
            if x not in no_duplicates_list:
                no_duplicates_list.append(x)
        return no_duplicates_list

# function to make dictionaries
def dictionary_maker(list1, list2):
    zip_iterator = zip(list1, list2)
    dictionary = dict(zip_iterator)
    return dictionary

# function to lemmatise words in lists
def lemmatise(list_of_words):
    lemmatised_words = []
    for word in list_of_words:
        lemmatised_words.append(lemmatizer.lemmatize(word))
    return lemmatised_words

# function user inputs veggies
mylist = []
mybasket = []
def user_inputs_veggies():
    print("Enter 3 veggies: ")
    for x in range(1,4):
        basket = input("%d " % x)
        mylist.append(basket.lower())
    for veg in lemmatise(mylist):
        if veg in veg_list:
            print(veg, "= got it")
            mybasket.append(veg)
        else:
            print(veg, "= we don't have it")

# function user inputs quantities (NOT USED FOR GETTING RECIPES YET)
veg_quantity = {}
def user_input_quantity():
    for x in mybasket:
        # Ask for the quantity, until it's correct
        while True:
            # Quantity?
            quantity = input("%s grams " % x)
            # Is it an integer?
            try:
                int(quantity)
                break
            # No...
            except ValueError:
                # Is it a float?
                try:
                    float(quantity)
                    break
                # No...
                except ValueError:
                    print("Please, use numbers in grams only")
        # If it's valid, add it
        veg_quantity[x] = quantity
    return veg_quantity

# CODE
# Extracting lists from pandas columns
veg_list = list_maker(df_veg['Vegetables'])
food_list = list_maker(df_all["Food"])
allergy_list = list_maker(df_all["Allergy"])
recipe_titles_list = list_maker(df_rec['Title'])
ingredients_list = list_maker(df_rec['Ingredients'])
users_id_list = list_maker(df_rat["userId"])
recipes_id_list = list_maker(df_rat["recipeId"])
ratings_list = list_maker(df_rat["rating"])

# Lower case in lists
veg_list = lower_case(veg_list)
food_list = lower_case(food_list)
allergy_list = lower_case(allergy_list)
#recipe_titles_list = lower_case(recipe_titles_list)
ingredients_list = lower_case(ingredients_list)

# Dictionaries
food_allergy_dictionary = dictionary_maker(food_list, allergy_list)
recipe_titles_ingredients_dictionary = dictionary_maker(recipe_titles_list, ingredients_list)
recipes_id_ratings_dictionary = dictionary_maker(recipes_id_list, ratings_list)

# User inputs veggies
user_inputs_veggies()            
if mybasket == []:
    print("Your basket is empty")
else:
    print("Here's what we have", mybasket)

# User inputs quantities
user_input_quantity() 

# REST OF THE CODE (Still to change...)
# USER INPUTS ALLERGIES (NOT USED FOR GETTING RECIPES YET)
print("Any allergies or intolerances? Please enter them here or leave it blank. \n")
print("Please, specify if you have allergy or intolerance for generic terms \n")
print("(e.g. 'nut allergy', 'gluten allergy', but not for 'strawberry' or 'strawberries'): ")

# add allergies in the list
myallergies = []

# empty basket to break
basket = " "

# indefinite iteration over not empty basket
while basket != "":
    
    # over input
    basket = input()
    
    # if input = num
    if basket.isnumeric() == True:
        
        # then print you don't want num 
        print("No numbers, please")
        
    # otherwise if it's a word 
    elif basket.isnumeric() == False: 
        
        # and the basket is not empty
        if basket != "":
            
            # append allergies to my list
            myallergies.append(basket)

my_allergies = lower_case(myallergies)            
my_allergies = lemmatise(myallergies)
my_allergies = no_duplicates(my_allergies)

for al in my_allergies:
    if al in food_allergy_dictionary.keys() or al in food_allergy_dictionary.values():
        print("You said: ", al)
    else:
        print(al, ", got it, I will update my database")
        
# OUTPUT = RECIPES BASED ON USER'S VEGGIES
# RegEx to find matches
import re
recipe_titles_list = []
recipe_title_to_matched_ingredient_list_dict_with_duplicates = {}
recipes_ingredients = {}
recipes = []


input_vegetable_list = mybasket
recipe_title_to_ingredient_list_dict = recipe_titles_ingredients_dictionary

for input_vegetable in input_vegetable_list:
     for recipe_title in recipe_title_to_ingredient_list_dict:
        ingredient_list_string = recipe_title_to_ingredient_list_dict[ recipe_title ]
        
        # df not perfect, values looked like list but it was a string... 
        # with eval is list of lists
        ingredient_list = eval(ingredient_list_string)
        
        for ingredient in ingredient_list:
            find = re.search(input_vegetable, ingredient)  
        
            if find:
                recipe_titles_list.append( recipe_title )
                
                if recipe_title in recipe_title_to_matched_ingredient_list_dict_with_duplicates:
                    
                    recipe_title_to_matched_ingredient_list_dict_with_duplicates[recipe_title].append(input_vegetable)
                else:
                    recipe_title_to_matched_ingredient_list_dict_with_duplicates[recipe_title] = [input_vegetable]
                
# duplicates removed                
for key, value in recipe_title_to_matched_ingredient_list_dict_with_duplicates.items():
    recipes_ingredients[key] = list(set(value))  
print("\n")

for recipe_title in recipe_titles_list:
    if recipe_title not in recipes:
        recipes.append(recipe_title)
        
print("These are all the recipes that contain : ", mybasket)
print("\n")
index = 1
for recipe in recipes:
    print(index, recipe)
    index += 1

import random    
recipes_ingredients_items = list(recipes_ingredients.items())
random.shuffle(recipes_ingredients_items)
recipes_ingredients = dict(recipes_ingredients_items)

index = 1
for recipe in recipes_ingredients.items():
    index += 1
    
recipes = {}
i_have_processed_these_already = []

for vegetable in input_vegetable_list:
    for key, values in recipes_ingredients.items():
        if vegetable in values:
            if not key in i_have_processed_these_already:
                if not vegetable in recipes:
                    i_have_processed_these_already.append(key)
                    recipes[vegetable] = key
                    
import pprint
print("\n")
print("I would recommend you to try these recipes: \n")
pprint.pprint(recipes)
print("\n")
print("Here you can see the ingredients for the recipes selected: \n")
for recipe in recipes.values():
    for recipeT, ingredient in recipe_title_to_ingredient_list_dict.items():
        if recipe in recipeT:
            print(recipeT, "\n",ingredient, "\n")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Enter 3 veggies: 
1 onions
2 garlic
3 parsnips
onion = got it
garlic = got it
parsnip = got it
Here's what we have ['onion', 'garlic', 'parsnip']
onion grams 1
garlic grams 1
parsnip grams 200
Any allergies or intolerances? Please enter them here or leave it blank. 

Please, specify if you have allergy or intolerance for generic terms 

(e.g. 'nut allergy', 'gluten allergy', but not for 'strawberry' or 'strawberries'): 
Strawberries
strawberries
Strawberry
strawberries
fish allergy
cashew
nut allergy
lactose
lactose intolerance

You said:  strawberry
You said:  fish allergy
You said:  cashew
You said:  nut allergy
You said:  lactose
You said:  lactose intolerance


These are all the recipes that contain :  ['onion', 'garlic', 'parsnip']


1 Spinach, sweet potato & lentil dhal
2 Sesame parsnip & wild rice tabbouleh
3 Veggie yaki udon
4 Roasted red pepper & tomato soup with ricotta
5 Veggie okonomiyaki
6 Spicy ‘vedgeree’
7 Artichoke & aubergine rice
8 Creamy tomato risotto
9 Italian borl